In [1]:
import pulp

class InventoryOptimization:
    def __init__(self, T, d, K, c, h, p, I0, SS):
        self.T = T                  # 計画期間
        self.d = d                  # 需要予測リスト
        self.K = K                  # 発注固定費用
        self.c = c                  # 仕入れ価格
        self.h = h                  # 保管コスト
        self.p = p                  # 在庫切れペナルティ
        self.I0 = I0                # 初期在庫
        self.SS = SS                # 安全在庫水準
        self.model = pulp.LpProblem("Inventory Optimization", pulp.LpMinimize)
    
    def define_variables(self):
        self.x = {t: pulp.LpVariable(f"x_{t}", lowBound=0, cat='Integer') for t in range(self.T)}
        self.I = {t: pulp.LpVariable(f"I_{t}", lowBound=0, cat='Integer') for t in range(self.T)}
        self.S = {t: pulp.LpVariable(f"S_{t}", lowBound=0, cat='Integer') for t in range(self.T)}
        self.y = {t: pulp.LpVariable(f"y_{t}", cat='Binary') for t in range(self.T)}
    
    def set_objective(self):
        self.model += pulp.lpSum(self.K * self.y[t] + self.c * self.x[t] + self.h * self.I[t] + self.p * self.S[t] for t in range(self.T))
    
    def add_constraints(self):
        for t in range(self.T):
            if t == 0:
                self.model += self.I[t] == self.I0 + self.x[t] - self.d[t] + self.S[t]
            else:
                self.model += self.I[t] == self.I[t-1] + self.x[t-1] - self.d[t] + self.S[t]
            self.model += self.I[t] >= self.SS
            self.model += self.x[t] <= 1000 * self.y[t]  # M = 1000 (十分大きな数)

    def optimize(self):
        self.define_variables()
        self.set_objective()
        self.add_constraints()
        self.model.solve()
        return pulp.value(self.model.objective), [(t+1, self.x[t].value(), self.I[t].value(), self.S[t].value()) for t in range(self.T)]

T = 6
d = [100, 120, 80, 110, 90, 130]
K, c, h, p = 100000, 1000, 300, 2000
I0, SS = 50, 10

optimizer = InventoryOptimization(T, d, K, c, h, p, I0, SS)
total_cost, results = optimizer.optimize()

print(f"総コスト: {total_cost:.2f}")
print("\n期間ごとの結果:")
print("期 発注量 在庫水準 在庫切れ量")
for result in results:
    print(f"{result[0]:2d} {result[1]:7.0f} {result[2]:8.0f} {result[3]:10.0f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/nago/dev/graph-generate/env/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/65157913457e4d39964a4834195bf1f5-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/65157913457e4d39964a4834195bf1f5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 137 RHS
At line 156 BOUNDS
At line 181 ENDATA
Problem MODEL has 18 rows, 24 columns and 41 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 569000 - 0.00 seconds
Cgl0003I 0 fixed, 12 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 11 rows, 22 columns (22 integer (5 of which binary)) and 33 elements
Cutoff increment increased from 1e-05 to 99.9999
Cbc0012I Integer solution of 923000 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc001

/home/nago/dev/graph-generate/env/lib/python3.10/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
